In [1]:
import numpy as np
import tensorflow as tf
import sklearn as sk
import sklearn.linear_model as sklm
import sklearn.metrics as skmt
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io
import h5py
import sys
import gc

sys.path.append('Metric/')
sys.path.append('../Visualization/')
sys.path.append('../Data_Preprocessing//')
from Metric import *
from Visualization import *
from Data_Extractor import *
from sklearn.externals import joblib

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# settings
classifier_type = 'LR'
path_image_dir = "../Data/090085/"
path_road_mask = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif"
path_model_dir = "./Segmentation-Central-Pixel/Result/motor_trunk_pri_sec_tert_uncl_track/sklearn/"
model_name = "sk-SGD_weight_m5_0_p1_e15_r1"
size = 8
step = 1

block_size = 128

use_norm = True
if model_name.find("_G") > 0:
    norm = "Gaussian"
elif model_name.find("_m") > 0:
    norm = "mean"
else:
    norm = None
    use_norm = False
    
image_list = ['090085_20170531.h5',
              '090085_20160528.h5',
              '090085_20150526.h5',
              '090085_20140523.h5',
              '090085_20130520.h5']

In [3]:
class Classifier ():
    def __init__(self, path, classifier_type):
        assert classifier_type in set(['LR', 'FCN'])
        self.classifier_type = classifier_type
        if classifier_type == 'LR':
            self.classifier = joblib.load(path)
            self.pos_idx = int(np.where(self.classifier.classes_ == 1)[0])
            self.predict = lambda patch: self.classifier.predict_proba(patch.reshape((1, -1)))[0,self.pos_idx]

        else: # FCN
            saver = tf.train.Saver()
            sess = tf.InteractiveSession()
            saver.restore(sess, path) # "/tmp/model.ckpt"
            
            graph = tf.get_default_graph()
            self.x = graph.get_tensor_by_name("input/x:0")
            self.prob_out = graph.get_tensor_by_name("prob_out/prob_out:0")
            
            self.pos_idx = 1
            self.predict = lambda patch: self.prob_out.eval(feed_dict={x: patch.transpose((0, 2, 3, 1)), 
                                                                       is_training: False})[self.pos_idx]

In [4]:
# re-load classifier
if classifier_type == 'LR':
    is_valid = lambda patch: (patch != -9999).all()
    
else:
    assert classifier_type == 'FCN'
    is_valid = lambda patch: ((patch==-9999).sum() / np.prod(np.array(patch.shape))) < 1/100,

classifier = Classifier(path_model_dir + model_name, classifier_type)

In [ ]:
path_pred_in_time = path_model_dir + "Time-Series/" + model_name + '.h5'
print(path_pred_in_time)

for image_name in image_list:
    gc.collect()
    
    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()

    # check prediction not existing
    date = image_name.split('.')[0].split('_')[-1]
    h5f = h5py.File(path_pred_in_time, 'w')
    if date in set([pred for pred in h5f.items()]):
        print(image_path, 'already predicted, skip')
        h5f.close()
        continue
    h5f.close()

    # construct extractor
    data_extractor = Pred_Data_Extractor(raw_image, step=step, size=size,
                                         normalization=norm, is_valid=is_valid)
    
    # pred on image
    pred_road = np.zeros(shape=raw_image[0].shape)
    print(pred_road.shape, image_name, date)
    if classifier_type == 'LR':
        for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=use_norm):
            pred_road[coord[0]+int(size/2),coord[1]+int(size/2)] = classifier.predict(patch)
    else: # FCN
        for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=use_norm):
            pred_road[coord[0]:coord[0]+size, coord[1]:coord[1]+size] += classifier.predict(patch)[:,1]
            
    # save prediction
    h5f = h5py.File(path_pred_in_time, 'w')
    h5f.create_dataset(name=date, data=pred_road)
    h5f.close()

./Segmentation-Central-Pixel/Result/motor_trunk_pri_sec_tert_uncl_track/sklearn/Time-Series/sk-SGD_weight_m5_0_p1_e15_r1.h5
mu =  [ 6.15251388e-05  1.31409052e-05  3.72616606e-05  8.35797834e-05
 -1.94599085e-05 -9.25174121e-05  1.31593428e-04]
(7961, 8091) 090085_20170531.h5 20170531
mu =  [ 1.37939874e-04 -1.59259309e-04 -1.82118851e-04  8.99532199e-05
 -1.81662886e-05  4.14640237e-05  7.96866500e-05]
(7961, 8091) 090085_20160528.h5 20160528


Analyze pred along the time

In [ ]:
def get_patch(coord, mask, size):
    return mask1[coord[0]:coord[0]+size, coord[1]:coord[1]+size]

In [ ]:
# load road mask
road_mask = skimage.io.imread(path_road_mask)
road_proba = dict{}

for image_name in image_list:
    gc.collect()

    # restore prediction 
    date = image_name.split('.')[0].split('_')[-1]
    h5f.open(path_pred_in_time, 'r')
    pred_road = np.array(h5f[date])
    h5f.close()

    # load raw image
    image_path = path_image_dir + image_name
    h5f = h5py.File(image_path, 'r')
    raw_image = np.array(h5f['scene'])
    h5f.close()
    
    # construct extractor
    data_extractor = Pred_Data_Extractor(raw_image, step=block_size, size=block_size,
                                         normalization=None, is_valid=lambda patch: True)
    assert not (date in road_proba)
    road_proba[date] = dict{}
    for coord, patch in data_extractor.iterate_raw_image_patches_with_coord(norm=False):
        sub_valid_mask = patch[0][0] != -9999
        sub_road_mask  = get_patch(coord, road_mask, block_size) == 1
        sub_valid_road_mask = np.logical_and(sub_valid_mask, sub_road_mask)
        
        assert not (coord in road_proba[date])
        road_proba[date][coord] = get_patch(coord, pred_road, block_size)[np.where(sub_valid_road_mask)].sum()

In [ ]:
# get coord_list
coord_list = sorted(road_proba[date].keys())
for image_path in image_list:
    date = image_path.split('.')[0].split('_')[-1]
    image_path = path_image_dir + image_path
    assert ( coord_list == sorted(road_proba[date].keys()) ).all()

# get date_list
date_list = sorted(road_proba.keys())

# construct matrix [date][coord]
sorted_road = np.array([[road_proba[date][coord] for coord in coord_list]
                        for date in date_list])

# plot proba by date
for row, coord in zip((sorted_road.T), coord_list):
    plt.plot(row, label=str(coord))
plt.legend()
plt.show()

In [ ]:
show_pred_prob_with_raw(raw_image, pred_road, pred_weight=0.3, figsize=(50,50), 
                        show_plot=True)
#                         save_path=model_path + 'Analysis/prob_road_on_raw - 0_1.png')

Analyze log pred

In [ ]:
log_pred = -np.log(-pred_road + 1 + 1e-7)
print(log_pred.min(), log_pred.max(), log_pred.mean())

norm_log_pred = (log_pred - log_pred.min()) / (log_pred.max()-log_pred.min())
print(norm_log_pred.min(), norm_log_pred.max(), norm_log_pred.mean())

plt.hist(x=norm_log_pred.flatten(), bins=100, histtype='step')
plt.show()
plt.close()
plt.hist(x=norm_log_pred[np.where(norm_log_pred>0)].flatten(), bins=100, histtype='step')
plt.show()
plt.close()

In [ ]:
path_road = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/road_mask.tif"
road_mask = skimage.io.imread(path_road)

In [ ]:
show_pred_road_against_raw(raw_image, norm_log_pred, road_mask, threshold=0.40, figsize=(150,150), show_plot=False,
                        save_path="123")
gc.collect()

In [ ]:
plt.imshow(np.array([[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]]), cmap='hot')
plt.show()
plt.figure(figsize=(100,100))
plt.imshow(pred_road, cmap=plt.get_cmap('hot'))
plt.show()
plt.close()

In [ ]:
plt.imshow(np.array([[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]]), cmap='hot')
plt.show()
plt.figure(figsize=(100,100))
plt.imshow(np.log(pred_road + 1e-9), cmap='hot')
plt.show()
plt.close()